In [1]:
import pandas as pd

import wandb

wandb.login()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: yw580 (pico-lm). Use `wandb login --relogin` to force relogin


True

In [2]:
TINY_BASE = "pico-lm/pico-decoder/lne20rc1"
SMALL_BASE = "pico-lm/pico-decoder/kzcalg17"

TINY_RELORA = "pico-lm/pico-relora/6vhmv4aj"
SMALL_RELORA = "pico-lm/pico-relora/5urfvs2j"

In [3]:
api = wandb.Api()

baseline_tiny = api.run(TINY_BASE)
relora_tiny = api.run(TINY_RELORA)
baseline_small = api.run(SMALL_BASE)
relora_small = api.run(SMALL_RELORA)

In [4]:
bt_history = pd.DataFrame(baseline_tiny.scan_history())
rt_history = pd.DataFrame(relora_tiny.scan_history())
bs_history = pd.DataFrame(baseline_small.scan_history())
rs_history = pd.DataFrame(relora_small.scan_history())

In [5]:
step = 20_000
for gpus, history in [(16, bt_history), (16, bs_history), (4, rt_history), (4, rs_history)]:
    step_row = history[history["trainer/global_step"] == step]

    if not step_row.empty:
        # '_runtime' is seconds since start
        runtime_up_to_step = step_row["_runtime"].values[0] / 3600
        gpu_hours = runtime_up_to_step * gpus
        print(f"Runtime: {runtime_up_to_step:.2f} hours = {gpu_hours:.2f} GPU.")
    else:
        print(f"Step {step} not found in run history")

Runtime: 52.96 hours = 847.37 GPU.
Runtime: 61.29 hours = 980.56 GPU.
Runtime: 183.17 hours = 732.70 GPU.
Runtime: 215.87 hours = 863.47 GPU.
